# Algorytmy Genetyczne: ćwiczenie 1

## 1. Opis/Cel ćwiczenia

Celem ćwiczenia jest zapoznanie się z podstawowymi etapami działania algorytmu genetycznego (AG) oraz zbadanie wpływu parametrów:
- prawdopodobieństwa krzyżowania jednopunktowego $ p_k $,
- prawdopodobieństwa mutacji $ p_m $,

na szybkość i jakość zbieżności wyników optymalizacji dla zadanej funkcji:
$$
f(x) = 0.2\sqrt{x} \;+\; 2 \sin\bigl(2\pi \cdot 0.02 \cdot x\bigr) \;+\; 5, \quad x \in \langle 0, 255 \rangle.
$$

W ramach zadania należy:
1. Zaimplementować algorytm genetyczny, który będzie poszukiwał maksymalnej wartości powyższej funkcji.
2. Zbadać wpływ różnych wartości $ p_k $ i $ p_m $ na zbieżność algorytmu (przedstawić wykresy i omówić wyniki).
3. Powtórzyć obliczenia dla liczby chromosomów (rozmiaru populacji) równej 200 oraz 50, a następnie porównać uzyskane rezultaty.
4. Skomentować otrzymane wyniki oraz przedstawić wnioski.


## 2. Wstęp teoretyczny

### 2.1 Podstawy algorytmów genetycznych

Algorytmy genetyczne (AG) są metodami optymalizacji inspirowanymi procesem ewolucji biologicznej. Wykorzystują mechanizmy:
- **Selekcji** (ang. _selection_),
- **Krzyżowania** (ang. _crossover_),
- **Mutacji** (ang. _mutation_),

aby iteracyjnie ulepszać rozwiązania (zwane chromosomami lub osobnikami) w kierunku optymalnego (lub zbliżonego do optymalnego) rozwiązania postawionego problemu.

#### 2.1.1 Kodowanie i dekodowanie
W klasycznym podejściu do algorytmów genetycznych przyjmuje się, że zmienna (lub zmienne) są reprezentowane binarnie. W tym zadaniu:
- Zakres $ x \in \langle 0, 255 \rangle $ można przedstawić na 8-bitowym chromosomie (ponieważ $ 2^8 = 256 $).

Po zakodowaniu $ x $ w postaci binarnej (np. `10101010`), konieczne jest dekodowanie tego ciągu na liczbę całkowitą z przedziału $\langle 0, 255 \rangle$$ w celu obliczenia wartości funkcji $ f(x) $.

#### 2.1.2 Selekcja
W algorytmach genetycznych selekcja ma na celu wybranie osobników o najlepszej wartości funkcji przystosowania (ang. _fitness_) do dalszej reprodukcji. Popularne metody selekcji:
- **Selekcja ruletkowa** (ang. _roulette wheel selection_),
- **Selekcja turniejowa** (ang. _tournament selection_),
- **Selekcja rankingowa** (ang. _ranking selection_).

W każdym przypadku chodzi o to, by osobniki o wyższej wartości funkcji przystosowania miały większą szansę na przejście do kolejnego pokolenia.

#### 2.1.3 Krzyżowanie (crossover)
Krzyżowanie jednopunktowe (ang. _single-point crossover_) polega na wybraniu losowego punktu przecięcia w chromosomie i wymianie części genów pomiędzy dwoma rodzicami. Przykład dla chromosomów 8-bitowych:

Rodzic A: 1010 | 1010 Rodzic B: 0011 | 1100 ^ punkt podziału (np. pomiędzy 4. a 5. genem) Potomstwo A': 1010 1100 Potomstwo B': 0011 1010

Prawdopodobieństwo krzyżowania $ p_k $) określa, jak często dochodzi do krzyżowania pary rodziców.


#### 2.1.4 Mutacja (mutation)
Mutacja w kontekście AG polega na odwróceniu (inwersji) bitu w chromosomie z pewnym prawdopodobieństwem $ p_m $. Jeśli $ p_m $ jest zbyt niskie, populacja może utknąć w optimum lokalnym; jeśli za wysokie – algorytm może zachowywać się zbyt losowo i mieć problemy ze stabilną zbieżnością.

#### 2.1.5 Różnice między algorytmami genetycznymi a metodami konwencjonalnymi
- **Operowanie na ciągach kodowych** (binarnych lub innych), a nie bezpośrednio na wartościach zmiennych.
- **Równoległe działanie na populacji** (wielu punktach) zamiast na pojedynczych punktach.
- **Poszukiwanie metodą próbkowania** (losowe reguły wyboru, brak konieczności analizy gradientów, itp.).
- **Stochastyczne reguły** decydujące o ewolucji (selekcja, krzyżowanie, mutacja).


## 3. Przebieg zadania

### 3.1 Implementacja algorytmu genetycznego

In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt

# Funkcja celu
def f(x):
    return 0.2 * np.sqrt(x) + 2 * np.sin(2 * np.pi * 0.02 * x) + 5

# Parametry
POP_SIZE = 200  # liczba chromosomów w populacji
GENS = 200      # liczba pokoleń
p_c = 0.7       # prawdopodobieństwo krzyżowania (przykładowa wartość)
p_m = 0.1       # prawdopodobieństwo mutacji (przykładowa wartość)
CHROM_LENGTH = 8  # liczba bitów na chromosom (dla zakresu 0-255)

def initialize_population(pop_size, chrom_length=8):
    # Generujemy losowe chromosomy binarne
    # Każdy chromosom jest tablicą bitów (0 lub 1)
    return [np.random.randint(0, 2, chrom_length).tolist() for _ in range(pop_size)]

def decode(chrom):
    # Chromosom to ciąg bitów; dekodujemy na liczbę całkowitą
    # np. chrom = [1,0,1,0,1,0,1,0] -> x = 170 (w systemie dziesiętnym)
    value = 0
    for bit in chrom:
        value = (value << 1) | bit
    return value

def fitness(chrom):
    x = decode(chrom)
    return f(x)

def selection(pop):
    # Przykład: selekcja ruletkowa
    fit_values = [fitness(ind) for ind in pop]
    total_fit = sum(fit_values)
    # Losujemy punkt na "ruletce"
    pick = random.uniform(0, total_fit)
    current = 0
    for i, val in enumerate(fit_values):
        current += val
        if current > pick:
            return pop[i]
    return pop[-1]

def crossover(parent1, parent2):
    # Krzyżowanie jednopunktowe
    if random.random() < p_c:
        point = random.randint(1, CHROM_LENGTH - 1)
        child1 = parent1[:point] + parent2[point:]
        child2 = parent2[:point] + parent1[point:]
    else:
        child1, child2 = parent1[:], parent2[:]
    return child1, child2

def mutate(chrom):
    for i in range(len(chrom)):
        if random.random() < p_m:
            chrom[i] = 1 - chrom[i]  # inwersja bitu
    return chrom

# Główna pętla algorytmu
def genetic_algorithm(pop_size=POP_SIZE, gens=GENS, p_c=0.7, p_m=0.1):
    population = initialize_population(pop_size, CHROM_LENGTH)
    best_values = []
    avg_values = []

    for g in range(gens):
        new_population = []
        # Zapisujemy statystyki
        fit_vals = [fitness(ind) for ind in population]
        best_values.append(np.max(fit_vals))
        avg_values.append(np.mean(fit_vals))

        # Tworzymy nową populację
        while len(new_population) < pop_size:
            parent1 = selection(population)
            parent2 = selection(population)
            child1, child2 = crossover(parent1, parent2)
            child1 = mutate(child1)
            child2 = mutate(child2)
            new_population.append(child1)
            new_population.append(child2)

        population = new_population[:pop_size]

    # Ostatnie obliczenie statystyk
    fit_vals = [fitness(ind) for ind in population]
    best_values.append(np.max(fit_vals))
    avg_values.append(np.mean(fit_vals))

    return best_values, avg_values

# Przykładowe wywołanie i wykres

In [ ]:
if __name__ == "__main__":
    p_c = 0.7
    p_m = 0.1
    best, avg = genetic_algorithm(POP_SIZE, GENS, p_c, p_m)

    plt.figure(figsize=(8,5))
    plt.plot(best, label="Najlepsza wartość f(x)")
    plt.plot(avg, label="Średnia wartość f(x)")
    plt.title(f"AG - pk={p_c}, pm={p_m}, PopSize={POP_SIZE}")
    plt.xlabel("Pokolenie")
    plt.ylabel("Wartość funkcji przystosowania")
    plt.legend()
    plt.grid(True)
    plt.show()

## 3.2 Eksperymenty z różnymi wartościami $ p_k $ i $ p_m $
1. Wybór wartości $ p_k $$: np. $ \{0.5, 0.6, 0.7, 0.8, 1.0\} $.
2. Wybór wartości $$ p_m $$: np. $$ \{0.0, 0.01, 0.06, 0.1, 0.2, 0.3, 0.5\} $.

Dla każdej pary $ (p_k, p_m) $ należy uruchomić powyższy algorytm, zapisać/wyświetlić uzyskiwane krzywe:
- Najlepsza wartość funkcji przystosowania w danym pokoleniu,
- Średnia wartość funkcji przystosowania w danym pokoleniu.
-
Następnie można uzupełnić tabelę z uzyskanymi średnimi wartościami w ostatnim pokoleniu (lub np. najlepszą wartością po 200 pokoleniach) dla każdej kombinacji $ p_k $ i $ p_m $.

### 3.3 Powtórzenie eksperymentu dla populacji o rozmiarze 50

W celu zbadania wpływu rozmiaru populacji na zbieżność algorytmu, zmieniamy parametr POP_SIZE = 50 w kodzie i ponownie wykonujemy obliczenia dla tych samych kombinacji $ p_k $ i $ p_m $. Wyniki (najlepsze i średnie wartości funkcji) porównujemy z rezultatami uzyskanymi dla populacji o rozmiarze 200.

# 4. Obserwacje
Na podstawie uzyskanych wykresów oraz wartości w tabelach można wyciągnąć następujące obserwacje:

1. Wpływ $ p_k $
- Dla niskich wartości $ p_k $ (np. 0.5) algorytm rzadziej dokonuje rekombinacji materiału genetycznego, co może spowalniać zbieżność.
- Dla wartości $p_k $ bliskich 1.0 populacja jest często krzyżowana, co zazwyczaj sprzyja szybszemu przeszukiwaniu przestrzeni rozwiązań.
- Zbyt wysokie $ p_k $ w połączeniu z wysokim $ p_m $ może jednak wprowadzać dużą losowość.

2. Wpływ $ p_m $
- Zbyt mała mutacja (np. $ p_m = 0 $) może spowodować, że populacja bardzo szybko zbiegnie się do lokalnego optimum i nie będzie w stanie się z niego wydostać.
- Umiarkowane wartości (np. $ 0.01 \le p_m \le 0.1 $) sprzyjają znalezieniu globalnego maksimum, zapewniając jednocześnie stabilność procesu.
- Zbyt wysoka mutacja (np. $ p_m = 0.5 $) wprowadza tak dużą losowość, że algorytm może „błądzić” i mieć trudności z osiągnięciem wysokich wartości.

3. Wpływ rozmiaru populacji

- Przy większej populacji (np. 200 chromosomów) algorytm ma większą różnorodność początkową i większą szansę na znalezienie najlepszego rozwiązania w krótszej liczbie pokoleń.
- Przy mniejszej populacji (np. 50 chromosomów) algorytm może szybciej konwergować, ale częściej wpada w minima lokalne (lub w danym przypadku – może nie osiągnąć tak wysokiej wartości maksymalnej, jak przy populacji większej).

4. Ostateczne wartości maksymalne
- Dla większości kombinacji $ p_k $ i $ p_m $ można zaobserwować, że po kilkudziesięciu (np. 50–80) pokoleniach następuje ustabilizowanie się populacji w okolicach pewnej wartości maksymalnej (np. ok. 10.0 – 10.2, w zależności od przebiegu).


# 5. Wnioski

1. Optymalne zakresy $ p_k $ i $ p_m $
- Na podstawie przeprowadzonych eksperymentów można stwierdzić, że umiarkowane wartości $ p_m $ (zazwyczaj od 0.01 do 0.1) w połączeniu z wysokim $ p_k $ (np. 0.7–1.0) dają dobre wyniki z punktu widzenia szybkości i jakości zbieżności.

2. Zbyt mała lub zbyt duża mutacja
- Brak mutacji ($ p_m = 0 $) prowadzi do szybkiego „zamrożenia” populacji, co nie zawsze gwarantuje osiągnięcie globalnego maksimum.
- Zbyt duża mutacja ($ p_m $ w okolicach 0.5) powoduje nadmierną losowość i destabilizację procesu.

3. Rozmiar populacji
- Większa populacja (200 chromosomów) daje większe prawdopodobieństwo szybkiego odnalezienia globalnego maksimum, ale zwiększa też koszty obliczeniowe (konieczność oceny większej liczby osobników).
- Mniejsza populacja (50 chromosomów) może szybciej konwergować, ale z ryzykiem pominięcia najlepszego rozwiązania.

4. Różnice między AG a konwencjonalnymi metodami optymalizacji
- Operowanie na kodzie binarnym i równoległe przeszukiwanie wielu rozwiązań naraz,
- Zastosowanie losowych mechanizmów (selekcja, mutacja, krzyżowanie) umożliwia opuszczanie pułapek lokalnych,
- Brak wymogu znajomości gradientu czy innej analitycznej informacji o funkcji.

W praktyce algorytmy genetyczne są często stosowane, gdy przestrzeń rozwiązań jest duża i skomplikowana, a tradycyjne metody optymalizacji (np. gradientowe) zawodzą lub są trudne do zastosowania. W niniejszym ćwiczeniu zaobserwowaliśmy, że odpowiednio dobrane parametry $ p_k $ i $ p_m $ pozwalają efektywnie odnaleźć maksimum zadanej funkcji w rozsądnej liczbie pokoleń.
